In [1]:
from two_step_classification import *
print_version()

ran:  2023-09-14 08:43:45.597134
python:      3.10.11
sklearn:     1.2.2
tensorflow:  2.12.0
keras:       2.12.0
numpy:       1.23.5
pandas:      1.5.3


In [2]:
results: list[dict[str, ]] = []

In [3]:
# load data
data_names = ["models/kdd99_features/y_test_df.pkl",
              "models/kdd99_features/x_test_df.pkl",
              "models/kdd99_features/x_test-drop_25_df.pkl",
              "models/kdd99_features/x_test-drop+ae_30_df&activation=relu&epochs=5&batch_size=32.pkl",
              "models/kdd99_features/x_test-drop+ae_35_df&activation=relu&epochs=5&batch_size=32.pkl",
              "models/kdd99_features/x_test-drop+ae_40_df&activation=relu&epochs=5&batch_size=32.pkl",
              "models/kdd99_features/x_test+ae_43_df&activation=relu&epochs=5&batch_size=32.pkl",
              "models/kdd99_features/x_test+ae_48_df&activation=relu&epochs=5&batch_size=32.pkl",
              ]
lgb_names = ["models/lightgbm/lgb_dropped+ae_30_"]
y_test: pd.Series = pd.read_pickle()
y_test_binary: pd.Series = y_test.apply(lambda x: 0 if x == 1 else 1)
y_test_anomaly: pd.Series = pd.read_pickle("models/kdd99_features/y_test_dropped_mapped_series.pkl")

In [4]:
dir_names_1st = "models/logistic_regression_binary/"
dir_names_2nd = "models/logistic_regression_anomaly/"
model_names = []
os.listdir(dir_name2)
